# Objectives 
* Determine if logistic regression can be done with a non-gigantic amount of variables, and if so, which variables
* Find patterns in the data and visualize them

First, we import the necessary libraries and get a first look at the data.

In [1]:
import pandas as pd
# probably import matplotlib and numpy later, seaborn for heatmaps

In [2]:
df = pd.read_csv('hackathon-riot-data/hackathon-riot-data.csv', sep=';')
pd.set_option('display.max_columns', None)
df

Unnamed: 0                          gameDate      esportsPlatformId  \
0               0  2023-06-18 18:14:11.781000+00:00  ESPORTSTMNT01:3379581   
1               1  2023-02-16 01:59:23.961000+00:00  ESPORTSTMNT02:3150844   
2               2  2021-03-13 21:07:56.263000+00:00  ESPORTSTMNT02:1721741   
3               3  2021-04-10 11:56:11.980000+00:00  ESPORTSTMNT06:1680685   
4               4  2023-06-08 19:47:14.554000+00:00  ESPORTSTMNT06:2815912   
...           ...                               ...                    ...   
19741       19741  2023-07-26 15:59:21.957000+00:00  ESPORTSTMNT04:2689259   
19742       19742  2022-08-30 16:35:42.548000+00:00  ESPORTSTMNT05:3090546   
19743       19743         2023-04-11 23:15:16+00:00  ESPORTSTMNT01:3343374   
19744       19744  2023-07-15 19:10:14.307000+00:00  ESPORTSTMNT03:3187730   
19745       19745  2022-02-13 20:36:58.697000+00:00  ESPORTSTMNT03:2563015   

          gameVersion  NbWardsPlacedBlue  NbWardsPlacedRed  \
0      13.11.512.8126                 54                58   
1       13.3.491.6222                  0                 0   
2       11.5.361.5971                 77               143   
3       11.6.364.6814                 42                56   
4      13.10.509.8402                 71                70   
...               ...                ...               ...   
19741   13.14.522.821                104               106   
19742  12.15.458.1416                 43                42   
19743   13.5.495.8836                 65                62   
19744  13.13.518.1870                 54                66   
19745   12.3.422.2870                 68                59   

       NbControlWardsPlacedBlue  NbControlWardsPlacedRed  NbWardsKilledBlue  \
0                            15                       23                  9   
1                             0                        0                 19   
2                            36                       37                 38   
3                            23                       17                 23   
4                            38                       36                 22   
...                         ...                      ...                ...   
19741                        42                       63                 37   
19742                        18                       16                 13   
19743                        33                       50                 15   
19744                        48                       36                 25   
19745                        36                       32                 17   

       NbWardsKilledRed  NbControlWardsKilledBlue  NbControlWardsKilledRed  \
0                    12                        11                        7   
1                    37                        21                       21   
2                    29                        23                       15   
3                    13                        11                       14   
4                    26                        25                       21   
...                 ...                       ...                      ...   
19741                44                        37                       23   
19742                15                         6                        7   
19743                27                        22                       26   
19744                22                        12                       31   
19745                27                        18                       22   

       NbCampsSecuredBlue  NbCampsSecuredRed  NbCampsStolenBlue  \
0                       0                  0                  0   
1                       0                  0                  0   
2                       0                  0                  5   
3                       0                  0                  1   
4                       0                  0                  1   
...                   ...            

In [3]:
# going down list of columns to see what can be consolidated
df.columns.tolist()
#df.iloc[19000].to_frame().T to get a row

['Unnamed: 0',
 'gameDate',
 'esportsPlatformId',
 'gameVersion',
 'NbWardsPlacedBlue',
 'NbWardsPlacedRed',
 'NbControlWardsPlacedBlue',
 'NbControlWardsPlacedRed',
 'NbWardsKilledBlue',
 'NbWardsKilledRed',
 'NbControlWardsKilledBlue',
 'NbControlWardsKilledRed',
 'NbCampsSecuredBlue',
 'NbCampsSecuredRed',
 'NbCampsStolenBlue',
 'NbCampsStolenRed',
 'NbScuttlesBlue',
 'NbScuttlesRed',
 'NbRiftHeraldsBlue',
 'NbRiftHeraldsRed',
 'NbDragonsBlue',
 'NbDragonsRed',
 'NbBaronsBlue',
 'NbBaronsRed',
 'NbEldersBlue',
 'NbEldersRed',
 'NbTowersBlue',
 'NbTowersRed',
 'NbPlatesBlue',
 'NbPlatesRed',
 'OuterTopBlueTimer',
 'OuterMidBlueTimer',
 'OuterBotBlueTimer',
 'InnerTopBlueTimer',
 'InnerMidBlueTimer',
 'InnerBotBlueTimer',
 'BaseTopBlueTimer',
 'BaseMidBlueTimer',
 'BaseBotBlueTimer',
 'Nexus1MidBlueTimer',
 'Nexus2MidBlueTimer',
 'OuterTopRedTimer',
 'OuterMidRedTimer',
 'OuterBotRedTimer',
 'InnerTopRedTimer',
 'InnerMidRedTimer',
 'InnerBotRedTimer',
 'BaseTopRedTimer',
 'BaseMidRed

We want to use logistic regression, a binary classifier, to predict which team wins. Therefore, rows without data in the "winner" column can't be used.

In [9]:
df['winner'].value_counts(dropna=False)

winner
blue    10427
red      9318
NaN         1
Name: count, dtype: int64

Luckily, we only have one row missing the winner label.

In [19]:
missing_label_row = (df['winner'].isna() == True)
df.drop(index=df[missing_label_row].index, inplace=True)
df.reset_index(drop=True, inplace=True)

We can reduce the 198 columns by consolidating. For example, we are given the gold of top, jg, mid, AD, and sup at 10 minutes for the blue side and the red side. These can be added into 2 columns. The same can be done for gold differences, vision scores, damage dealt, damage taken, etc. Consolidation will also make modeling easier. There may be nuances to consider, such as possibly keeping the gold of adc separate since esports games favor funneling the adc, especially if more competitive tournaments tend to have longer matches (adc shines late game), but for now we simply want an mvp.

In [4]:
# redundant column same thing as index
df.drop(columns=['Unnamed: 0'], inplace=True)

Before consolidating columns, we need to check for missing values and address them appropriately, see if any can be replaced or if the row they belong to must be dropped.

In [5]:
df[['VisionScoreTopBlue', 'VisionScoreJgBlue', 'VisionScoreMidBlue', 'VisionScoreADBlue', 'VisionScoreSupBlue']].isnull().sum()

VisionScoreTopBlue    1
VisionScoreJgBlue     1
VisionScoreMidBlue    1
VisionScoreADBlue     1
VisionScoreSupBlue    1
dtype: int64

In [6]:
#filt = df['VisionScoreTopBlue']
#filt = (df[‘colName’] conditional), df[filt] returns dataframe
nan_rows = df[df[['VisionScoreTopBlue', 'VisionScoreJgBlue', 'VisionScoreMidBlue', 'VisionScoreADBlue', 'VisionScoreSupBlue']].isna().any(axis=1)]
nan_rows

gameDate      esportsPlatformId     gameVersion  \
7872  2023-06-29 23:55:26.709000+00:00  ESPORTSTMNT02:3214601  13.12.518.1641   

      NbWardsPlacedBlue  NbWardsPlacedRed  NbControlWardsPlacedBlue  \
7872                 47                56                        28   

      NbControlWardsPlacedRed  NbWardsKilledBlue  NbWardsKilledRed  \
7872                       15                 20                11   

      NbControlWardsKilledBlue  NbControlWardsKilledRed  NbCampsSecuredBlue  \
7872                        14                        8                   0   

      NbCampsSecuredRed  NbCampsStolenBlue  NbCampsStolenRed  NbScuttlesBlue  \
7872                  0                  2                 0               0   

      NbScuttlesRed  NbRiftHeraldsBlue  NbRiftHeraldsRed  NbDragonsBlue  \
7872              0                  0                 0              0   

      NbDragonsRed  NbBaronsBlue  NbBaronsRed  NbEldersBlue  NbEldersRed  \
7872             0             0            0             0            0   

      NbTowersBlue  NbTowersRed  NbPlatesBlue  NbPlatesRed  OuterTopBlueTimer  \
7872             0            8             1            8                NaN   

      OuterMidBlueTimer  OuterBotBlueTimer  InnerTopBlueTimer  \
7872                NaN                NaN                NaN   

      InnerMidBlueTimer  InnerBotBlueTimer  BaseTopBlueTimer  \
7872                NaN                NaN               NaN   

      BaseMidBlueTimer  BaseBotBlueTimer  Nexus1MidBlueTimer  \
7872               NaN               NaN                 NaN   

      Nexus2MidBlueTimer  OuterTopRedTimer  OuterMidRedTimer  \
7872                 NaN          1055.006          1048.438   

      OuterBotRedTimer  InnerTopRedTimer  InnerMidRedTimer  InnerBotRedTimer  \
7872           911.552          1193.864          1191.993          1430.789   

      BaseTopRedTimer  BaseMidRedTimer  BaseBotRedTimer  Nexus1MidRedTimer  \
7872              NaN         1358.273         1453.812                NaN   

      Nexus2MidRedTimer  DragonSoulTimer  DragonSoulType  DragonSoulTaker  \
7872                NaN              NaN             NaN              NaN   

      GoldTopBlue10  GoldJgBlue10  GoldMidBlue10  GoldADBlue10  GoldSupBlue10  \
7872         3307.0        3144.0         3266.0        3386.0         2163.0   

      GoldTopRed10  GoldJgRed10  GoldMidRed10  GoldADRed10  GoldSupRed10  \
7872        2674.0       2939.0        2851.0       3088.0        2011.0   

      GoldDiff10Top  GoldDiff10Jg  GoldDiff10Mid  GoldDiff10AD  GoldDiff10Bot  \
7872          633.0         205.0          415.0         298.0          152.0   

      XPDiff10Top  XPDiff10Jg  XPDiff10Mid  XPDiff10AD  XPDiff10Bot  \
7872       -324.0      -264.0        109.0       619.0        105.0   

      BlueKills10  BlueAssists10  BlueDeaths10  BlueTotalGold10  \
7872          2.0            4.0           0.0          15266.0   

      BlueDragonKills10  BlueTowerKills10  RedKills10  RedAssists10  \
7872                0.0               0.0         0.0           0.0   

      RedDeaths10  RedTotalGold10  RedDragonKills10  RedTowerKills10  \
7872          2.0         13563.0               1.0              0.0   

      GoldTopBlue15  GoldJgBlue15  GoldMidBlue15  GoldADBlue15  GoldSupBlue15  \
7872         5386.0        5267.0         7006.0        5959.0         3426.0   

      GoldTopRed15  GoldJgRed15  GoldMidRed15  GoldADRed15  GoldSupRed15  \
7872        4288.0       4560.0        4305.0       5444.0        3054.0   

      GoldDiff15Top  GoldDiff15Jg  GoldDiff15Mid  GoldDiff15AD  GoldDiff15Bot  \
7872         1098.0         707.0         2701.0         515.0          372.0   

      XPDiff15Top  XPDiff15Jg  XPDiff15Mid  XPDiff15AD  XPDiff15Bot  \
7872       -163.0      -338.0       1202.0      1015.0        319.0   

      BlueKills15  BlueAssists15  BlueDeaths15  BlueTotalGold15  \
7872          7.0           16.0           1.0          27044.0   

   